In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio

In [141]:
dataObjs = curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)
allModels = []
tickersSeen = []
for item in dataObjs:
    if item["IS_SHARPE SLIPPAGE"] > 0.7:
        model = item["model"]
        if model.numberOfPredictors() > 6:
            print(model.targetTicker, model.getHash(), item["IS_SHARPE SLIPPAGE"], item["IS_SHARPE DIFFERENCE SLIPPAGE"], item["IS_BETA"])
            allModels.append(model)
            if model.targetTicker not in tickersSeen:
                tickersSeen.append(model.targetTicker)
        

SMH a1a918b5b238537e1ef42455652f04a2d9586f8bfa0a28e852be4d23 0.7624352491212067 0.028445511456217876 0.436142582900723
KBE 9016b9e865e0585a8d6f29fad6700010897307e7ae38acd069b7427b 0.7417734388757505 0.12921447773009354 0.21715084053760447
KBE e0f24d9c117d94a504a26f2cc8de6e34099b78d75e9b8a79f0596748 0.7093481206397568 0.14355824495522684 0.11842070566901326
EWY 009e190789f632aa9df114a7e4d6e6dadfe8e69cd50f13a7bc2f7672 0.8647789208041518 0.15949649045643088 0.4399641766475613
SLV 15bceada618bfb11879bdc724eef33dcd350617ff8930e6c8b9da2e2 0.8452414124117282 0.3578646368260958 0.1983039566450171
XOP 1bf1ff0268474e21f2fa9873df1606bf6dbe390bbb149ba1c621c3ef 0.8067939660315397 0.5288099446247327 0.13683719124212754
XOP 05ffc8a72f0be314fcdfb3bc95c137d9aae2adc4c779c91fe97ccd6d 0.7055614964184781 0.5611473776681195 0.12946302053546338
SLV 3ae9aa36b51d0a528b582363e2a5eb4093f081bcf0039483eb999978 1.0704517504223032 0.6576323869746924 0.17282565453178658
OIH d6437661b905b27d128be345c4aacd7ee87b215c9e5

In [142]:
len(allModels)

11

In [143]:
tickersSeen

['SMH', 'KBE', 'EWY', 'SLV', 'XOP', 'OIH', 'XME']

In [113]:
import random
factorToTrade = tickersSeen[random.randint(0, len(tickersSeen) - 1)]
factorToTrade

'XME'

In [114]:
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


['KBE', 'EWY', 'SLV', 'XOP', 'OIH', 'XME']
KBE
SLV
XME
XOP
OIH
EWY
ATTEMPTING PULL 9016b9e865e0585a8d6f29fad6700010897307e7ae38acd069b7427b
((((('VO', 10, None, None, 2), 'KBE', 22, 5, 1.5, 0.1, 10), (('SSO', 30, None, None, 2), 'KBE', 10, 5, 0.5, 0.1, 10), 5, 'OR'), ((('VWO', 40, None, None, 4), 'KBE', 10, 5, 0.3, 0.01, 10), (('VUG', 8, None, None, 3), 'KBE', 10, 5, 0.7, 0.01, 10), 5, 'AND'), 5, 'OR'), ((('ILF', 7, 5, None, 4), 'KBE', 44, 5, 2.0, 0.1, 10), ((('DIA', 6, 18, None, 2), 'KBE', 10, 5, 0.7, 0.2, 10), (('IJH', 7, 17, 5, 4), 'KBE', 22, 5, 1.5, 0.01, 10), 5, 'OR'), 5, 'AND'), 5, 'OR')
ATTEMPTING PULL e0f24d9c117d94a504a26f2cc8de6e34099b78d75e9b8a79f0596748
((((('EWG', 23, 3, None, 4), 'KBE', 5, 5, 0.5, 0.01, 10), (('IAU', 27, 19, None, 4), 'KBE', 22, 5, 1.0, 0.05, 10), 5, 'AND'), ((('VO', 10, None, None, 2), 'KBE', 22, 5, 1.5, 0.1, 10), (('SSO', 30, None, None, 2), 'KBE', 10, 5, 0.5, 0.1, 10), 5, 'OR'), 5, 'OR'), (((('SSO', 30, None, None, 2), 'KBE', 10, 5, 0.7, 0.01, 10), (('

In [115]:
cleanedReturns = modelReturns.dropna()
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.dropna()
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item

In [116]:
def historicalWeightsToTickerAllocations(historicalWeights, algorithmPredictions, modelsInPortfolio):
    aggregatePredictions = algorithmPredictions.dropna()
    allocationsToStore = []
    historicalAllocations = None
    scaledHistoricalAllocations = None
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.targetTicker not in netPosition:
                netPosition[model.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.targetTicker] += weights[model.getHash()] * aggregatePredictions.loc[historicalWeights.index[i]][model.getHash()]
        thisDf = pd.DataFrame([netPosition], index=[historicalWeights.index[i]])
        if historicalAllocations is None:
            historicalAllocations = thisDf
        else:
            historicalAllocations = pd.concat([historicalAllocations, thisDf])
        
        totalCapitalUsed = sum([abs(netPosition[ticker]) for ticker in netPosition])
        scaledNetPosition = {}
        for ticker in netPosition:
            scaledNetPosition[ticker] = netPosition[ticker] * 1.0/totalCapitalUsed
        
        thisDf = pd.DataFrame([scaledNetPosition], index=[historicalWeights.index[i]])
        if scaledHistoricalAllocations is None:
            scaledHistoricalAllocations = thisDf
        else:
            scaledHistoricalAllocations = pd.concat([scaledHistoricalAllocations, thisDf])
    
    return historicalAllocations, scaledHistoricalAllocations
            
            

In [133]:
import empyrical
def getLimitedDataForPortfolio(historicalWeights, historicalPredictions, modelsUsed, factorToTrade, joinedData):
    
    normalTickerAllocationsTable, scaledTickerAllocationsTable = historicalWeightsToTickerAllocations(historicalWeights, historicalPredictions, modelsUsed)
    
    tickerAllocationsTable = scaledTickerAllocationsTable
    rawTickerPerformance = portfolioGeneration.calculatePerformanceForTable(tickerAllocationsTable, tickerAllocationsTable.columns, joinedData)

    rawAlgoPerformance = pd.DataFrame(rawTickerPerformance.apply(lambda x:sum(x), axis=1), columns=["Algo Return Without Commissions"])

    tickerPerformance, algoPerformance, algoTransactionCost =  portfolioGeneration.calculatePerformanceForAllocations(tickerAllocationsTable, joinedData)

    benchmark = factorToTrade
    factorReturn = dataAck.getDailyFactorReturn(benchmark, joinedData)
    factorReturn.columns = ["Factor Return (" + benchmark + ")"]
    algoPerformance.columns = ["Algo Return"]
    
    tickersUsed = []
    for mod in modelsUsed:
        tickersUsed.append(mod.targetTicker)
    
    for ticker in tickersUsed:
        thisFactorReturn = dataAck.getDailyFactorReturn(ticker, joinedData)
        thisFactorReturn.columns = ["Factor Return (" + ticker + ")"]
        alpha, beta = empyrical.alpha_beta(algoPerformance, thisFactorReturn)
        print(ticker, beta)

    alpha, beta = empyrical.alpha_beta(algoPerformance, factorReturn)
    sharpe_difference = empyrical.sharpe_ratio(algoPerformance) - empyrical.sharpe_ratio(factorReturn)
    annualizedReturn = empyrical.annual_return(algoPerformance)[0]
    annualizedVolatility = empyrical.annual_volatility(algoPerformance)
    
    ##AUTOMATICALLY TAKES SLIPPAGE INTO ACCOUNT
    return {
        "benchmark":factorToTrade,
        "alpha":alpha,
        "beta":beta,
        "sharpe difference":sharpe_difference,
        "annualizedReturn":annualizedReturn,
        "annualizedVolatility":annualizedVolatility,
        "sharpe":empyrical.sharpe_ratio(algoPerformance),
        "free return":annualizedReturn - annualizedVolatility
    }
    


In [134]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(10, len(algoColumns)), replace= False)


In [135]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            
            cov = (aggregateReturns[i-windowSize:i]).cov()
        try:
            weights = hrp.getHRP(cov, corr)
        #     display(weights)
        #     display(aggregateReturns[i+windowSize:i+windowSize+1])
            todayReturn = aggregateReturns[i:i+1] * weights
        #     display(todayReturn)
            sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
            hrpReturns = pd.concat([hrpReturns, sumReturn])
            thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
            historicalWeights = pd.concat([historicalWeights, thisWeights])
        except:
            print("FAILED:",i)
        i += 1
    return hrpReturns, historicalWeights

In [136]:
def storeDiscoveredPortfolio(models, portfolioType, benchmark, IS_DATA, OOS_DATA):
    description = "AUTO GENERATED"
    portfolioType = "HRP FULL"
    seenTickers = []
    
    allHashes = []
    for model in models:
        allHashes.append(model.getHash())
        if model.targetTicker not in seenTickers:
            seenTickers.append(model.targetTicker)
        
    ##SORT SO ENSURE SAME PORTFOLIO NOT CREATED TWICE
    allHashes = sorted(allHashes)
    
    portfolioString = str(allHashes) + benchmark + description + portfolioType
    portfolioHash = hashlib.sha224(portfolioString.encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.discoveredPortfolioModels, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark,
                "portfolioType":portfolioType,
                "startedTrading":curveTreeDB.getToday()
            }
            
            for k in IS_DATA:
                toUpload["IS_"+ k] = IS_DATA[k]
            
            for k in OOS_DATA:
                toUpload["OOS_"+ k] = OOS_DATA[k]
                
            for ticker in seenTickers:
                toUpload[ticker] = True
            
            toUpload["TICKERS TRADED"] = len(seenTickers)
            
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.discoveredPortfolios, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            return portfolioHash
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [137]:
def getWeightingForAlgos(allModels, columns):
    countPerTicker = {}
    hashes = {}
    for mod in allModels:
        hashes[mod.getHash()] = mod.targetTicker
        if mod.targetTicker not in countPerTicker:
            countPerTicker[mod.targetTicker] = 0.0
        countPerTicker[mod.targetTicker] += 1.0
    weightsToSend = []
    for col in columns:
        weightsToSend.append(1.0/countPerTicker[hashes[col]])
        
    return [item/sum(weightsToSend) for item in weightsToSend]

In [138]:
def performPortfolioPerformanceEstimation(historicalPredictions, historicalReturns, factorToTrade, portfolioType, hashToModel, joinedData):
    returnWindows = [(0, historicalReturns[:450]), (450, historicalReturns)]
    historicalWeights = None
    for selectedReturns in returnWindows:
        startIndex = selectedReturns[0]
        returnWindow = selectedReturns[1]
        weightsSeen = None
        if portfolioType == "HRP FULL":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=False)
        elif portfolioType == "HRP WINDOW":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=True)
        elif portfolioType == "EW":
            weightsSeen = pd.DataFrame(returnWindow[startIndex:].apply(lambda x: [1.0/len(x) for item in x], axis=1), columns=returnWindow.columns.values)
        elif portfolioType == "EW By Ticker":
            weightArray = getWeightingForAlgos(allModels, returnWindow.columns)
            weightsSeen = pd.DataFrame(returnWindow[startIndex:].apply(lambda x: weightArray, axis=1), columns=returnWindow.columns.values)
            
        
        
        if historicalWeights is None:
            historicalWeights = weightsSeen
        else:
            historicalWeights = pd.concat([historicalWeights, weightsSeen])
        
        modelsUsed = []

        tickersSeen = {}

        for modelHash in historicalPredictions.columns:
            thisModel = hashToModel[modelHash]
            modelsUsed.append(thisModel)
        if startIndex == 0:
            scaledStats = getLimitedDataForPortfolio(historicalWeights,\
                                    historicalPredictions, modelsUsed, factorToTrade, joinedData)
            print(scaledStats)
            if scaledStats["sharpe difference"] < 0.0 or scaledStats["annualizedReturn"] < scaledStats["annualizedVolatility"]:
                return None, None
    trainStats = getLimitedDataForPortfolio(historicalWeights[:-252], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    testStats = getLimitedDataForPortfolio(historicalWeights[-252:], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    
    if trainStats["sharpe difference"] > 0.0 and trainStats["annualizedReturn"] > trainStats["annualizedVolatility"]:
        print(trainStats, testStats)
        storeDiscoveredPortfolio(modelsUsed, portfolioType, factorToTrade, trainStats, testStats)
    else:
        print("FAILED", trainStats)
    

In [139]:
## MP RUN  
types = ["EW", "HRP WINDOW", "HRP FULL", "EW By Ticker"]
def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        factorToTrade = hashToModel[selectedAlgorithms[random.randint(0, len(selectedAlgorithms) - 1)]].targetTicker
        
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
            
        portfolioType = types[random.randint(0, len(types) - 1)]
        print(factorToTrade, len(selectedAlgorithms), portfolioType)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, portfolioType, hashToModel, joinedData))
        p.start()
        runningP.append(p)

# In[ ]:


In [140]:
print("STARTING GENERATION")

##REMOVE BREAK TO DO FULL AUTO
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=0)

STARTING GENERATION
SLV 10 HRP WINDOW
XOP 0.393368811819
KBE 0.377807894237
XOP 0.393368811819
SLV 0.32529589118
SLV 0.32529589118
OIH 0.433162437408
XME 0.42453189041
SLV 0.32529589118
KBE 0.377807894237
EWY 0.444444838264
{'benchmark': 'SLV', 'alpha': 0.283928255787227, 'beta': 0.32529589118003166, 'sharpe difference': 1.3300614746836528, 'annualizedReturn': 0.43389136503348547, 'annualizedVolatility': 0.2552478387167792, 'sharpe': 1.5399350008521258, 'free return': 0.17864352631670627}
XOP 0.3976701324
KBE 0.38241628711
XOP 0.3976701324
SLV 0.316871668514
SLV 0.316871668514
OIH 0.436103447937
XME 0.424220346366
SLV 0.316871668514
KBE 0.38241628711
EWY 0.430568920279
XOP 0.385267254856
KBE 0.476317216361
XOP 0.385267254856
SLV 0.270555016094
SLV 0.270555016094
OIH 0.425040928992
XME 0.387968131616
SLV 0.270555016094
KBE 0.476317216361
EWY 0.675242555866
{'benchmark': 'SLV', 'alpha': 0.2821223880014777, 'beta': 0.31687166851446885, 'sharpe difference': 1.230325539366393, 'annualizedRe

KeyboardInterrupt: 